In [3]:
#Standard Imports für alle Notebooks

!pip3 install tira ir-datasets python-terrier

from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
import pyterrier as pt

ensure_pyterrier_is_loaded()
tira = Client()

pt_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')

index = tira.pt.index('ir-lab-sose-2024/tira-ir-starter/Index (tira-ir-starter-pyterrier)', pt_dataset)

In [4]:
#Erstellen von costum Retrieval Systemen

def create_index(documents, stemmer):
    indexer = pt.IterDictIndexer("/tmp/index", overwrite=True, meta={'docno': 100, 'text': 20480}, stemmer=stemmer, stopwords=None)
    
    index_ref = indexer.index(documents)
    return pt.IndexFactory.of(index_ref)



LemurKrovetzStemmer_index = create_index(pt_dataset.get_corpus_iter(), 'LemurKrovetzStemmer')
WeakPorterStemmer_index = create_index(pt_dataset.get_corpus_iter(), 'WeakPorterStemmer')
PorterStemmer_index = create_index(pt_dataset.get_corpus_iter(), 'PorterStemmer')
EnglishSnowballStemmer_index = create_index(pt_dataset.get_corpus_iter(), 'EnglishSnowballStemmer')
SnowballStemmer_index = create_index(pt_dataset.get_corpus_iter(), 'SnowballStemmer')
TRv2WeakPorterStemmer_index = create_index(pt_dataset.get_corpus_iter(), 'TRv2WeakPorterStemmer')
TRv2PorterStemmer_index = create_index(pt_dataset.get_corpus_iter(), 'TRv2PorterStemmer')

XSqrA_M	 = pt.BatchRetrieve(index, wmodel="XSqrA_M")
XSqrA_M_LemurKrovetzStemmer	 = pt.BatchRetrieve(LemurKrovetzStemmer_index, wmodel="XSqrA_M")
XSqrA_M_WeakPorterStemmer	 = pt.BatchRetrieve(WeakPorterStemmer_index, wmodel="XSqrA_M")
XSqrA_M_PorterStemmer	 = pt.BatchRetrieve(PorterStemmer_index, wmodel="XSqrA_M")
XSqrA_M_EnglishSnowballStemmer	 = pt.BatchRetrieve(EnglishSnowballStemmer_index, wmodel="XSqrA_M")
XSqrA_M_SnowballStemmer	 = pt.BatchRetrieve(SnowballStemmer_index, wmodel="XSqrA_M")
XSqrA_M_TRv2WeakPorterStemmer	 = pt.BatchRetrieve(TRv2WeakPorterStemmer_index, wmodel="XSqrA_M")
XSqrA_M_TRv2PorterStemmer	 = pt.BatchRetrieve(TRv2PorterStemmer_index, wmodel="XSqrA_M")

ir-lab-sose-2024/ir-acl-anthology-20240504-training documents:   0%|          | 0/126958 [00:00<?, ?it/s]

10:10:53.222 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - TermPipeline object org.terrier.terms.LemurKrovetzStemmer not found: java.lang.ClassNotFoundException: org.terrier.terms.LemurKrovetzStemmer


java.lang.ClassNotFoundException: org.terrier.terms.LemurKrovetzStemmer
	at java.base/jdk.internal.loader.BuiltinClassLoader.loadClass(BuiltinClassLoader.java:581)
	at java.base/jdk.internal.loader.ClassLoaders$AppClassLoader.loadClass(ClassLoaders.java:178)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:527)
	at java.base/java.lang.Class.forName0(Native Method)
	at java.base/java.lang.Class.forName(Class.java:398)
	at org.terrier.utility.ApplicationSetup.getClass(ApplicationSetup.java:416)
	at org.terrier.structures.indexing.Indexer.load_pipeline(Indexer.java:323)
	at org.terrier.structures.indexing.Indexer.init(Indexer.java:197)
	at org.terrier.structures.indexing.classical.BasicIndexer.<init>(BasicIndexer.java:183)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingCo

10:12:56.345 [ForkJoinPool-5-worker-3] WARN org.terrier.structures.indexing.Indexer - TermPipeline object org.terrier.terms.SnowballStemmer not found: java.lang.NoSuchMethodException: org.terrier.terms.SnowballStemmer.<init>(org.terrier.terms.TermPipeline)


java.lang.NoSuchMethodException: org.terrier.terms.SnowballStemmer.<init>(org.terrier.terms.TermPipeline)
	at java.base/java.lang.Class.getConstructor0(Class.java:3349)
	at java.base/java.lang.Class.getConstructor(Class.java:2151)
	at org.terrier.structures.indexing.Indexer.load_pipeline(Indexer.java:324)
	at org.terrier.structures.indexing.Indexer.init(Indexer.java:197)
	at org.terrier.structures.indexing.classical.BasicIndexer.<init>(BasicIndexer.java:183)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
	at org.terrier.python.ParallelIndexer$3.apply(ParallelIndexer.java:126)
	at org.terrier.python.ParallelIndexer$3.apply(Parall

In [5]:
#Evaluation

pt.Experiment(
    [XSqrA_M, XSqrA_M_TRv2PorterStemmer, XSqrA_M_TRv2WeakPorterStemmer, XSqrA_M_SnowballStemmer,XSqrA_M_EnglishSnowballStemmer, XSqrA_M_LemurKrovetzStemmer, XSqrA_M_PorterStemmer, XSqrA_M_WeakPorterStemmer],
    pt_dataset.get_topics(),
    pt_dataset.get_qrels(),
    ["ndcg_cut.10", "recip_rank", "recall_100", "P_10"],
    names = ["XSqrA_M", "XSqrA_M_TRv2PorterStemmer", "XSqrA_M_TRv2WeakPorterStemmer", "XSqrA_M_SnowballStemmer", "XSqrA_M_EnglishSnowballStemmer", "XSqrA_M_LemurKrovetzStemmer", "XSqrA_M_PorterStemmer", "XSqrA_M_WeakPorterStemmer"]
)

Download from the Incubator: https://files.webis.de/data-in-production/data-research/tira-zenodo-dump-preparation/ir-lab-sose2024/ir-acl-anthology-20240504-truth.zip?download=1
	This is only used for last spot checks before archival to Zenodo.


Download: 100%|██████████| 29.6k/29.6k [00:00<00:00, 1.38MiB/s]

Download finished. Extract...
Extraction finished:  /root/.tira/extracted_datasets/ir-lab-sose-2024/ir-acl-anthology-20240504-training/


There are multiple query fields available: ('text', 'title', 'query', 'description', 'narrative'). To use with pyterrier, provide variant or modify dataframe to add query column.
10:14:42.456 [main] ERROR org.terrier.terms.BaseTermPipelineAccessor - TermPipeline object org.terrier.terms.SnowballStemmer not found
java.lang.NoSuchMethodException: org.terrier.terms.SnowballStemmer.<init>(org.terrier.terms.TermPipeline)
	at java.base/java.lang.Class.getConstructor0(Class.java:3349)
	at java.base/java.lang.Class.getConstructor(Class.java:2151)
	at org.terrier.terms.BaseTermPipelineAccessor.<init>(BaseTermPipelineAccessor.java:71)
	at org.terrier.querying.ApplyTermPipeline.load_pipeline(ApplyTermPipeline.java:90)
	at org.terrier.querying.ApplyTermPipeline.getPipeline(ApplyTermPipeline.java:150)
	at org.terrier.querying.ApplyTermPipeline.process(ApplyTermPipeline.java:162)
	at org.terrier.querying.LocalManager.runSearchRequest(LocalManager.java:895)
10:15:07.777 [main] ERROR org.terrier.terms

,name,ndcg_cut.10,recip_rank,recall_100,P_10
0,XSqrA_M,0.448211,0.650289,0.592214,0.400000
1,XSqrA_M_TRv2PorterStemmer,0.420330,0.666520,0.587545,0.370588
2,XSqrA_M_TRv2WeakPorterStemmer,0.204095,0.337838,0.320920,0.170588
3,XSqrA_M_SnowballStemmer,0.122536,0.201694,0.222242,0.094118
4,XSqrA_M_EnglishSnowballStemmer,0.405770,0.655383,0.568348,0.355882
5,XSqrA_M_LemurKrovetzStemmer,0.122536,0.201694,0.222242,0.094118
6,XSqrA_M_PorterStemmer,0.410890,0.666412,0.568662,0.358824
7,XSqrA_M_WeakPorterStemmer,0.203491,0.336778,0.318626,0.169118
